In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def generate_random_data(num_entries):
    """Genera datos aleatorios para el DataFrame."""
    return {
        'Year': np.random.choice(range(2000, 2021), num_entries),
        'Country': np.random.choice(['Country A', 'Country B', 'Country C'], num_entries),
        'Biofuel Consumption': np.random.rand(num_entries) * 1000,
        'Solar Consumption': np.random.rand(num_entries) * 1000,
        'Wind Consumption': np.random.rand(num_entries) * 1000,
        'Renewable Energy Consumption': np.random.rand(num_entries) * 1000
    }

def create_dataframe(data):
    """Crea un DataFrame a partir de los datos generados."""
    return pd.DataFrame(data)

def load_dataset(path):
    """Carga el dataset desde un archivo CSV."""
    try:
        dataset = pd.read_csv(path)
        print("Dataset cargado exitosamente.")
        return dataset
    except FileNotFoundError:
        print("El archivo no fue encontrado.")
        return None

def dataset_info(dataset):
    """Muestra información básica sobre el dataset."""
    print("\nInformación del dataset:")
    print(dataset.info())
    print("\nPrimeras filas del dataset:")
    print(dataset.head())

def rename_columns(dataset):
    """Renombra las columnas del DataFrame."""
    column_mapping = {
        'Year': 'Año',
        'Country': 'País',
        'Biofuel Consumption': 'Consumo total de biocombustibles',
        'Solar Consumption': 'Consumo total de energía solar',
        'Wind Consumption': 'Consumo total de energía eólica',
        'Renewable Energy Consumption': 'Consumo total de energías renovables'
    }

    # Renombrar las columnas
    dataset.rename(columns=column_mapping, inplace=True)
    
    # Verificar el cambio de nombres
    print("\nColumnas renombradas:")
    print(dataset.columns)
    
    # Guardar el DataFrame renombrado en un archivo CSV
    dataset.to_csv('data/energy_clean_dataset.csv', index=False)  # Guardar el DataFrame renombrado
    print("\nArchivo guardado como 'energy_clean_dataset.csv'.")

def plot_data(dataset):
    """Crea gráficos de los datos del dataset."""    
    plt.figure(figsize=(10, 5))
    sns.lineplot(data=dataset, x='Año', y='Consumo total de biocombustibles', hue='País')
    plt.title('Consumo Total de Biocombustibles por País a lo Largo de los Años')
    plt.xlabel('Año')
    plt.ylabel('Consumo Total de Biocombustibles')
    plt.legend(loc='upper right')
    plt.show()

# Ejemplo de uso
# Generación de datos aleatorios
random_data = generate_random_data(20)
df_random = create_dataframe(random_data)

# Cargar el dataset
dataset_path = 'D:\\Documents\\GitHub\\world-energy-consumption\\data\\owid_energy_data.csv'
dataset = load_dataset(dataset_path)

if dataset is not None:
    dataset_info(dataset)
    rename_columns(dataset)
    plot_data(dataset)


El archivo no fue encontrado.
